In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
from sklearn.linear_model import LinearRegression
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output

# 🧮 `DataProcessor` Class Overview

The `DataProcessor` class is responsible for **loading**, **cleaning**, and **enhancing** transaction data. This includes:

- Removing invalid or extreme data points (outliers)
- Calculating key financial metrics
- Preparing the dataset for time-based analysis

---

## 🧼 Data Cleaning Pipeline

### 🔁 `__init__(self, file_path)`

Initializes the processor by loading a CSV and calling `_clean_data()`.

---

### 🧹 `_clean_data(self)`

Applies all cleaning and transformation steps:
- Drops missing values
- Removes outliers using **category-specific IQR**
- Adds financial metrics (sales, cost, profit)
- Converts and sets the datetime index

---

### 🚫 `_remove_outliers(self)`

Removes outliers **per category** based on `quantity` and `selling_price`.

#### ❗ Why Remove Per Category?

1. **Different Scales:**  
   Prices and quantities vary drastically between product types.

2. **Context Sensitivity:**  
   Selling 1,000 pens is normal; 1,000 laptops is suspicious.

3. **Statistical Accuracy:**  
   Outlier detection is done using category-specific **IQR**:

   - $ \text{IQR} = Q3 - Q1 $
   - $ \text{Lower Bound} = Q1 - 1.5 \times \text{IQR} $
   - $ \text{Upper Bound} = Q3 + 1.5 \times \text{IQR} $

   Any record outside these bounds for **quantity or selling price** is removed.

---

### 💰 `_add_financial_metrics(self)`

Calculates key financial metrics for every transaction:

| Metric          | Formula                                                | Description                                    |
|------------------|--------------------------------------------------------|------------------------------------------------|
| **Total Sales**  | $ \text{Quantity} \times \text{Selling Price} $        | Revenue from product sales                     |
| **Total Cost**   | $ \text{Quantity} \times \text{Cost Price} $           | Total cost incurred on the products sold       |
| **Profit**       | $ \text{Total Sales} - \text{Total Cost} $             | Net earnings per transaction                   |

---

### 🗓️ `_prepare_datetime_index(self)`

- Converts the `date` column to datetime format
- Sets it as the DataFrame index
- Sorts the DataFrame by date

---


In [3]:
class DataProcessor:
    def __init__(self, file_path):
        self.df = pd.read_csv(file_path)
        self._clean_data()
        
    def _clean_data(self):
        """Perform all data cleaning and preprocessing steps"""
        self.df = self.df.dropna()
        self._remove_outliers()
        self._add_financial_metrics()
        self._prepare_datetime_index()
        
    def _remove_outliers(self):
        """Remove outliers using category-specific IQR method"""
        df_clean = self.df.copy()
        for category in self.df['category'].unique():
            cat_mask = self.df['category'] == category
            
            # Quantity outliers
            q1_qty = self.df[cat_mask]['quantity'].quantile(0.25)
            q3_qty = self.df[cat_mask]['quantity'].quantile(0.75)
            iqr_qty = q3_qty - q1_qty
            
            # Price outliers
            q1_price = self.df[cat_mask]['selling_price'].quantile(0.25)
            q3_price = self.df[cat_mask]['selling_price'].quantile(0.75)
            iqr_price = q3_price - q1_price
            
            # Create filters
            qty_mask = (
                (self.df['quantity'] >= (q1_qty - 1.5*iqr_qty)) & 
                (self.df['quantity'] <= (q3_qty + 1.5*iqr_qty))
            )
            price_mask = (
                (self.df['selling_price'] >= (q1_price - 1.5*iqr_price)) & 
                (self.df['selling_price'] <= (q3_price + 1.5*iqr_price))
            )
            
            # Apply filters
            df_clean = df_clean[~(cat_mask & (~qty_mask | ~price_mask))]
            
        self.df = df_clean.reset_index(drop=True)
        
    def _add_financial_metrics(self):
        """Calculate financial metrics for each transaction"""
        self.df['total_sales'] = self.df['quantity'] * self.df['selling_price']
        self.df['total_cost'] = self.df['quantity'] * self.df['cost_price']
        self.df['profit'] = self.df['total_sales'] - self.df['total_cost']
        self.df['profit_margin'] = self.df['profit'] / self.df['total_sales']
        
    def _prepare_datetime_index(self):
        """Convert and set datetime index"""
        self.df['date'] = pd.to_datetime(self.df['date'])
        self.df.set_index('date', inplace=True)
        self.df.sort_index(inplace=True)

In [4]:
# Load and inspect data
processor = DataProcessor("/Users/kushal/Documents/optimized_retail_sales_new.csv")
df = processor.df

# 📊 Sales Distribution by State and Category (2020–2024)

## 📝 Summary

This grouped bar chart visualizes **total sales** across U.S. states and product categories for the period 2020–2024.

---

### 🧭 Axes

- **X-axis**: U.S. states, sorted from **highest to lowest** total sales  
- **Y-axis**: Total Sales in dollars ($)

---

### 🔍 Key Insights

#### 1️⃣ **State Performance Ranking**  
> 🏆 Easily identify top and bottom performing states  
- Left-to-right bar ordering shows sales dominance at a glance  
- Highlights regional strengths and opportunities

---

#### 2️⃣ **Category Breakdown**  
> 🎨 Color-coded segmentation reveals category contribution  
- Each state bar is **stacked or grouped** by product category  
- Enables comparison of category dominance **within** and **across** states

---

#### 3️⃣ **Sales Magnitude**  
> 📈 Bar height = total sales volume  
- Hover tooltips display **exact dollar values**

---

This visualization supports **data-driven strategy** by combining geographic and product-level insights.


In [6]:
state_category_sales = df.groupby(['state', 'category'])['total_sales'].sum().reset_index()

fig1 = px.bar(
    state_category_sales,
    x='state',
    y='total_sales',
    color='category',
    title='Sales Distribution by State and Category (2020-2024)',
    labels={'total_sales': 'Total Sales ($)', 'state': 'State'},
    barmode='group',
    custom_data=['category']
)

fig1.update_traces(
    hovertemplate="<b>%{x}</b><br>Sales: $%{y:,.2f}",
    marker_line_color='black',
    marker_line_width=1.5
)

fig1.update_layout(
    title_x=0.5,
    title_y=0.95,
    xaxis={
        'categoryorder': 'total descending',
        'showgrid': True,
        'gridcolor': '#DDD',
        'showline': True,
        'linecolor': 'black',
        'mirror': True, 
        'linewidth': 2  
    },
    yaxis={
        'showgrid': True,
        'gridcolor': '#DDD',
        'showline': True,
        'linecolor': 'black',
        'mirror': True, 
        'linewidth': 2
    },
    hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial"),
    legend_title_text='Category',
    paper_bgcolor='white',
    legend=dict(title_font_size=14, font_size=12),
    margin=dict(l=50, r=50, t=50, b=50),
    font=dict(family="Arial, sans-serif"),
    plot_bgcolor='white',
    height=400,
    width=1100
)

# 📈 Monthly Sales Trend (2020–2024)

## 📝 Summary

This line chart with markers illustrates the **monthly total sales trend** over a five-year span (2020–2024).

---

### 🧭 Axes

- **X-axis**: Month (January to December, repeated across years)
- **Y-axis**: Total Sales in dollars ($)

---

### 🔍 Key Insights

#### 1️⃣ **Temporal Pattern Analysis**  
> 📅 Observe patterns in sales over time

- Detect **seasonal fluctuations**, such as peaks during holidays or end-of-quarter boosts  
- Identify **long-term growth or decline** trends  
- Markers emphasize **individual monthly values**, enhancing data granularity

---

This chart helps uncover **seasonality**, monitor **performance stability**, and inform **strategic timing** for campaigns or restocking.

In [8]:
time_series = df.resample('M')['total_sales'].sum().reset_index()

fig2 = px.line(
    time_series,
    x='date',
    y='total_sales',
    title='Monthly Sales Trend (2020-2024)',
    markers=True
)

fig2.update_traces(
    line=dict(width=2.5, color='#1f77b4'),
    hovertemplate="<b>%{x|%b %Y}</b><br>Sales: $%{y:,.2f}<extra></extra>",
    name="Total Sales"
)

fig2.update_layout(
    xaxis_title='Month',
    yaxis_title='Total Sales ($)',
    hovermode='x unified',
    xaxis=dict(
        showgrid=True,
        gridcolor='#DDD',
        showline=True,
        linecolor='black',
        mirror=True,  
        linewidth=2   
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='#DDD',
        showline=True,
        linecolor='black',
        mirror=True,  
        linewidth=2  
    ),
    plot_bgcolor='white',
    title_x=0.5,
    legend_title_text="Sales Trend",
    legend=dict(
        title_font_size=14,
        font_size=12,
        bordercolor='black',
        borderwidth=1
    ),
    margin=dict(l=50, r=50, t=60, b=50),
    height=400,
    width=1100
)

# 💰 Top 10 Products by Profit Margin (%)

## 📝 Summary

This horizontal bar chart displays the **top 10 products** with the highest **profit margin percentages**, providing insight into which products are the most financially efficient.

---

### 🧭 Axes

- **X-axis**: Profit Margin (%) — bar length represents percentage margin  
- **Y-axis**: Product names, ordered from **lowest to highest** margin

---

### 🔍 Key Insights

#### 1️⃣ **Margin Ranking**  
> 📈 Identify your most profitable products relative to sales

- Bars are sorted in **ascending order** — the product with the highest profit margin appears at the **bottom**  
- Hover tooltips reveal both margin and total sales, offering dual insight into performance

---

This visualization aids in **strategic pricing**, **resource allocation**, and **product portfolio optimization** based on profitability—not just revenue.


In [10]:
# Calculate profit percentage and get top products
product_stats = df.groupby('product_name').agg(
    total_profit=('profit', 'sum'),
    total_sales=('total_sales', 'sum')
).reset_index()

product_stats['profit_percentage'] = (product_stats['total_profit'] / product_stats['total_sales']) * 100
top_products = product_stats.nlargest(10, 'profit_percentage')

# Create horizontal bar chart
fig = px.bar(
    top_products,
    x='profit_percentage',
    y='product_name',
    orientation='h',
    title='Top 10 Products by Profit Margin (%)',
    labels={'profit_percentage': 'Profit Margin (%)', 'product_name': ''},
    text='profit_percentage',
    color_discrete_sequence=['#1f77b4']
)

fig.update_traces(
    marker_line_color='black',
    marker_line_width=1,
    hovertemplate="<b>%{y}</b><br>Margin: %{x:.1f}%<br>Total Sales: $%{customdata[0]:,.0f}<extra></extra>",
    customdata=top_products[['total_sales']],
    texttemplate='%{x:.1f}%'
)

fig.update_layout(
    yaxis={
        'categoryorder': 'total ascending',
        'showline': True,
        'linecolor': 'black',
        'linewidth': 2,
        'mirror': True
    },
    xaxis={
        'showgrid': True,
        'gridcolor': '#DDD',
        'showline': True,
        'linecolor': 'black',
        'linewidth': 2,
        'mirror': True,
        'range': [0, 100]  # Ensures 0-100% scale
    },
    plot_bgcolor='white',
    margin=dict(l=120, r=50, t=80, b=50),
    title_x=0.5,
    height=500,
    width=900
)

fig.show()

# 📊 Seasonal Sales Comparison by Product Category (2020–2024)

## 📝 Summary

This **grouped bar chart** compares total sales across different product categories, split by **season** (Summer, Fall, Winter, Rainy) and **year** from 2020 to 2024. It provides an intuitive view of how product demand shifts seasonally over multiple years.

---

### 🧭 Axes

- **X-axis**: Product Category  
- **Y-axis**: Total Sales in dollars ($)  
- **Color**: Season (e.g., Fall, Summer, Winter, Rainy)  
- **Facets**: One column per year (2020, 2021, 2022, 2023, 2024)

---

### 🔍 Key Insights

#### 1️⃣ **Seasonal Demand Shifts**  
> 🌦 Spot which product categories peak during each season

- Compare how seasonality impacts sales year over year  

---

#### 2️⃣ **Category Strengths**  
> 🏆 Identify consistent or rising product performers

- Highlight categories with reliable year-round or seasonal success  
- Detect emerging or declining categories in specific years  

---

#### 3️⃣ **Strategic Implications**  
> 📈 Guide promotions, inventory, and marketing

- Align sales strategy with **seasonal trends**  
- Inform data-backed planning and resource allocation  

---

This chart enhances your understanding of **category-level seasonality**, supporting smarter business decisions throughout the year.


In [12]:
# Aggregate total sales by year, category, and season
seasonal_sales = df.groupby(['year', 'category', 'season'])['total_sales'].sum().reset_index()

# Create the bar chart with faceting by year
fig = px.bar(
    seasonal_sales,
    x='category',
    y='total_sales',
    color='season',
    text='season',
    barmode='group',
    facet_col='year',
    title='Seasonal Sales Comparison by Product Category and Year',
    labels={'category': 'Product Category', 'total_sales': 'Total Sales($)', 'season': 'Season'},
    color_discrete_map={
        'Spring': '#F4A261',
        'Summer': '#2A9D8F',
        'Fall': '#E76F51',
        'Winter': '#264653'
    },
    height=600,
    width=1100
)

# Update traces and layout as before
fig.update_traces(
    marker_line_color='black',
    marker_line_width=1,
    hovertemplate='Category: %{x}<br>Season: %{text}<br>Sales: $%{y:,.2f}<extra></extra>'
)

fig.update_layout(
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(l=40, r=40, t=60, b=80),
    legend_title='Season',
    legend=dict(
        bgcolor='white',
        bordercolor='black',
        borderwidth=1,
        font_size=12,
        title_font_size=14
    ),
    shapes=[{
        'type': 'rect',
        'xref': 'paper',
        'yref': 'paper',
        'x0': 0, 'y0': 0,
        'x1': 1, 'y1': 1,
        'line': {'color': 'black', 'width': 2}
    }]
)

fig.show()

# 🌎 Profit Distribution by State and Season (2020–2024)

## 📝 Summary

This grouped bar chart displays **total profit by U.S. state**, broken down by **season** across the years 2020 to 2024. It highlights **seasonal performance patterns** within each state.

---

### 🧭 Axes

- **X-axis**: U.S. States — grouped by **Spring**, **Summer**, **Fall**, and **Winter**  
- **Y-axis**: Total Profit in dollars ($)

---

### 🔍 Key Insights

#### 1️⃣ **Seasonal Trends by State**  
> 📊 Compare how each season contributes to state-level profitability

- Bars are **grouped by season** within each state 

#### 2️⃣ **Performance Hotspots**  
> 🏆 Identify states and seasons with consistently high profitability

- States are **sorted by total profit** — making top-performing regions easy to spot  
- Useful for optimizing **regional marketing** and **seasonal inventory planning**

---

This visualization supports **seasonal strategy**, **regional targeting**, and **year-over-year profitability analysis**.


In [14]:
# Aggregate total profit by state and season
state_season_profit = df.groupby(['state', 'season'])['profit'].sum().reset_index()

# Create the combined bar chart
fig = px.bar(
    state_season_profit,
    x='state',
    y='profit',
    color='season',
    title='Profit Distribution by State and Season (2020-2024)',
    labels={'profit': 'Total Profit ($)', 'state': 'State'},
    barmode='group',
    color_discrete_map={
        'Spring': '#F4A261',
        'Summer': '#2A9D8F',
        'Fall': '#E76F51',
        'Winter': '#264653'
    },
    height=500,
    width=1000
)

# Customize traces
fig.update_traces(
    marker_line_color='black',
    marker_line_width=1,
    hovertemplate="<b>%{x}</b><br>Season: %{fullData.name}<br>Profit: $%{y:,.2f}<extra></extra>"
)

# Add black border on all 4 axes
fig.update_layout(
    xaxis={
        'categoryorder': 'total descending',
        'showline': True,
        'linecolor': 'black',
        'linewidth': 2,
        'mirror': True
    },
    yaxis={
        'title': 'Total Profit ($)',
        'showline': True,
        'linecolor': 'black',
        'linewidth': 2,
        'mirror': True
    },
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='Season',
    title_x=0.5,
    margin=dict(l=50, r=50, t=80, b=50)
)

fig.show()


# 💰 Profit Margin Distribution by Product Category

## 📝 Summary

This **bar chart** visualizes the **average profit margin** across different product categories. It helps assess profitability by showing how much profit is earned per dollar of sales in each category.

---

### 🧭 Axes

- **X-axis**: Product Category  
- **Y-axis**: Average Profit Margin (as a percentage)  
- **Bars**: Colored and labeled with percentage values for quick interpretation

---
### 🔍 Key Insights

#### 1️⃣ **Profitability Leaders**
> 🏆 **Footwear** leads with the highest average profit margin of **39.61%**, followed closely by **Furniture** (**34.62%**) and **Attire** (**34.44%**).

- These categories contribute significantly to profit, even if their sales volume is lower.  
- Consider investing in marketing or premium positioning for these categories to enhance overall profitability.

---

#### 2️⃣ **Underperforming Category**
> ⚠️ **Groceries** shows the lowest margin at just **15.19%**.

- Indicates a price-sensitive or highly competitive segment  
- May require better cost control or selective product focus to improve margin

---

#### 3️⃣ **Balanced Opportunity**
> ⚙️ **Sports** products show a healthy margin of **31.08%**.

- A reliable contributor to both sales and profits  
- Suitable for bundling, promotions, or upselling strategies

---

This chart provides insights into **category-level profitability**, helping guide smarter pricing, marketing, and inventory decisions.

In [16]:
# Calculate profit margin and average by category
df["profit_margin"] = df["profit"] / df["total_sales"]
category_margin = df.groupby("category")["profit_margin"].mean().reset_index()
fig_margin = px.bar(
    category_margin,
    x="category",
    y="profit_margin",
    title="Product Margin Distribution by Category",
    labels={"profit_margin": "Average Profit Margin", "category": "Product Category"},
    text_auto=".2%",  
    color_discrete_sequence=["#3B82F6"],
    height=500,
    width=1000
)
fig_margin.update_traces(
    marker_line_color='black',
    marker_line_width=1.2,
    hovertemplate="<b>%{x}</b><br>Profit Margin: %{y:.2%}<extra></extra>",
    textfont=dict(color='white', size=12)
)
fig_margin.update_layout(
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white',
    yaxis=dict(
        title="Average Profit Margin",
        showgrid=True,
        gridcolor='#EEE',
        linecolor='black',
        linewidth=2,
        mirror=True # Add right-side border
    ),
    xaxis=dict(
        showline=True,
        linecolor='black',
        linewidth=2,
        mirror=True # Add top border
    ),
    margin=dict(l=60, r=30, t=60, b=60)
)

fig_margin.show()

# 📦 Units Sold by Product Category and Season (2020–2024)

## 📝 Summary

This grouped bar chart displays the **total units sold** for each **product category**, broken down by **season** over the period 2020 to 2024. It helps analyze how demand varies across categories and time of year.

---

### 🧭 Axes

- **X-axis**: Product categories - grouped by **Spring**, **Summer**, **Fall**, and **Winter**  
- **Y-axis**: Total Units Sold (count of items)

---


### 🔍 Key Insights

#### 1️⃣ **Seasonal Sales Patterns**  
> 📊 Compare how sales volumes fluctuate by category and season

- Categories are **grouped by season**, making seasonal shifts easy to interpret
  
#### 2️⃣ **Demand-Driven Strategy**  
> 🧠 Use trends to inform stocking and promotional planning

- Product categories with **strong seasonal performance** can guide inventory and marketing

---

This visualization supports **sales forecasting**, **inventory management**, and **seasonal strategy development**.


In [18]:
# Aggregate quantity by category and season
category_season_qty = df.groupby(['category', 'season'])['quantity'].sum().reset_index()

# Create visualization
fig = px.bar(
    category_season_qty,
    x='category',
    y='quantity',
    color='season',
    title='Units Sold by Product Category and Season (2020-2024)',
    labels={'quantity': 'Units Sold', 'category': 'Product Category'},
    barmode='group',
    color_discrete_map={
        'Spring': '#F4A261',
        'Summer': '#2A9D8F',
        'Fall': '#E76F51',
        'Winter': '#264653'
    },
    height=500,
    width=1000
)

# Customize layout
fig.update_traces(
    marker_line_color='black',
    marker_line_width=1,
    hovertemplate="<b>%{x}</b><br>%{fullData.name}<br>Units Sold: %{y:,}<extra></extra>"
)

# Add black borders on all 4 axes
fig.update_layout(
    xaxis={
        'categoryorder': 'total descending',
        'showline': True,
        'linecolor': 'black',
        'linewidth': 2,
        'mirror': True
    },
    yaxis={
        'title': 'Total Units Sold',
        'showline': True,
        'linecolor': 'black',
        'linewidth': 2,
        'mirror': True
    },
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='Season',
    title_x=0.5,
    margin=dict(l=50, r=50, t=80, b=50)
)

fig.show()


# 🛍️ `RetailDashboard` Class Overview

The `RetailDashboard` class creates an interactive analytics dashboard for retail data using **JupyterDash**. It enables:

* Dynamic filtering by geographic, product, and metric parameters
* Real-time visualization updates
* Predictive modeling with performance tracking

---

## 🖥️ Dashboard Structure

### 🔌 `__init__(self, data)`
Initializes the dashboard with:
* JupyterDash app instance
* Data storage
* Layout setup
* Callback registration

---

## 🎨 Layout Configuration (`_setup_layout`)

### 1️⃣ **Control Panel**
* **Metric Selector**: Toggle between `sales` and `profit`
* **State Filter**: Geographic segmentation (default: "All")
* **Category Filter**: Product category focus
* **Product Filter**: Dynamic options based on category/state

### 2️⃣ **Visualization Area**
| Component           | Description                                  | ID              |
|---------------------|----------------------------------------------|-----------------|
| Scatter Plot        | Actual vs Predicted values                  | `scatter-plot`  |
| Time Series Plot    | Temporal performance tracking               | `time-plot`     |
| Model Metrics Card  | R² and MAE display                          | `model-metrics` |

---

## 🔄 Callback System (`_register_callbacks`)

### 🔄 `update_products(category)`
Updates product dropdown options based on selected category.

### 🔄 `update_plots(metric, state, category, product)`
Main callback that drives all visual updates:

1. **Data Filtering**  
   Applies state/category/product filters to dataset
   
2. **Feature Engineering**  
   * Drops non-predictive columns  
   * Applies one-hot encoding to categorical features

3. **Model Pipeline**  
   ```mermaid
   graph TD
     A[Train-Test Split] --> B[Feature Scaling]
     B --> C[Linear Regression]
     C --> D[Prediction]
     D --> E[Metric Calculation]
     E --> F[Visualization Rendering]
     
### Visual Outputs

- 📉 **Scatter Plot** — Compares actual vs predicted values with a trendline
- 📈 **Time Series Plot** — Shows actual and predicted values over time
- 📊 **Model Metrics** — Displays:
  - **R² Score**: Measures model accuracy
  - **MAE (Mean Absolute Error)**: Average absolute prediction error

### 🚀 Execution (run)
```python
▶️ run() 

In [20]:
class RetailDashboard:
    def __init__(self, data):
        self.app = JupyterDash(__name__)
        self.data = data
        self._setup_layout()
        self._register_callbacks()
        
    def _setup_layout(self):
        """Configure dashboard layout"""
        self.app.layout = html.Div([
            html.H1("Retail Sales Analytics Dashboard", 
                    style={'textAlign': 'center'}),
            
            html.Div([
                html.Div([
                    html.Label("Performance Metric:", 
                              style={'fontWeight': 'bold', 'marginBottom': '5px'}),
                    dcc.Dropdown(
                        id='metric',
                        options=[
                            {'label': 'Sales', 'value': 'sales'},
                            {'label': 'Profit', 'value': 'profit'}
                        ],
                        value='sales',
                        style={'width': '200px'}
                    )
                ], style={'marginRight': '20px'}),
                
                html.Div([
                    html.Label("Filter by State:", 
                              style={'fontWeight': 'bold', 'marginBottom': '5px'}),
                    dcc.Dropdown(
                        id='state',
                        options=[{'label': s, 'value': s} for s in ['All'] + self.data['state'].unique().tolist()],
                        value='All',
                        style={'width': '200px'}
                    )
                ], style={'marginRight': '20px'}),
                
                html.Div([
                    html.Label("Filter by Category:", 
                              style={'fontWeight': 'bold', 'marginBottom': '5px'}),
                    dcc.Dropdown(
                        id='category',
                        options=[{'label': c, 'value': c} for c in ['All'] + self.data['category'].unique().tolist()],
                        value='All',
                        style={'width': '200px'}
                    )
                ], style={'marginRight': '20px'}),
                
                html.Div([
                    html.Label("Filter by Product:", 
                              style={'fontWeight': 'bold', 'marginBottom': '5px'}),
                    dcc.Dropdown(
                        id='product',
                        options=[{'label': 'All', 'value': 'All'}],
                        value='All',
                        style={'width': '200px'}
                    )
                ])
            ], style={
                'padding': '20px',
                'display': 'flex',
                'justifyContent': 'center',
                'gap': '20px',
                'flexWrap': 'wrap'
            }),
            
            html.Div([
                dcc.Graph(
                    id='scatter-plot',
                    style={'height': '400px', 'marginBottom': '20px'}
                ),
                
                dcc.Graph(
                    id='time-plot',
                    style={'height': '400px', 'marginBottom': '20px'}
                ),
                
                html.Div(
                    id='model-metrics',
                    style={
                        'padding': '20px',
                        'fontFamily': 'Arial',
                        'border': '1px solid #ddd',
                        'borderRadius': '5px',
                        'marginTop': '20px',
                        'textAlign': 'center',
                        'backgroundColor': '#f9f9f9'
                    }
                )
            ], style={'padding': '20px'})
        ])
    
    def _register_callbacks(self):
        """Register all dashboard callbacks"""
        @self.app.callback(
            Output('product', 'options'),
            Input('category', 'value'))
        def update_products(selected_category):
            if selected_category == 'All':
                return [{'label': 'All', 'value': 'All'}]
            products = self.data[self.data['category'] == selected_category]['product_name'].unique()
            return [{'label': 'All', 'value': 'All'}] + [{'label': p, 'value': p} for p in products]

        @self.app.callback(
            [Output('scatter-plot', 'figure'),
             Output('time-plot', 'figure'),
             Output('model-metrics', 'children')],
            [Input('metric', 'value'),
             Input('state', 'value'),
             Input('category', 'value'),
             Input('product', 'value')])
        def update_plots(selected_metric, selected_state, selected_category, selected_product):
            filtered_df = self.data.copy()
            
            if selected_state != 'All':
                filtered_df = filtered_df[filtered_df['state'] == selected_state]
            if selected_category != 'All':
                filtered_df = filtered_df[filtered_df['category'] == selected_category]
            if selected_product != 'All':
                filtered_df = filtered_df[filtered_df['product_name'] == selected_product]
            
            target_col = 'profit' if selected_metric == 'profit' else 'total_sales'
            display_name = 'Profit' if selected_metric == 'profit' else 'Total Sales'
            
            features = filtered_df.drop(columns=[
                'transaction_id', 'product_id', 'product_name',
                'total_sales', 'total_cost', 'profit', 'is_high_sales'
            ])
            
            X = pd.get_dummies(features, columns=['state', 'season', 'category'])
            y = filtered_df[target_col]
            
            split_idx = int(len(filtered_df) * 0.8)
            X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
            y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
            
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            model = LinearRegression()
            model.fit(X_train_scaled, y_train)
            predictions = model.predict(X_test_scaled)
            
            r2 = r2_score(y_test, predictions)
            mae = mean_absolute_error(y_test, predictions)
            
            results_df = pd.DataFrame({
                'date': filtered_df.iloc[split_idx:].index,
                'actual': y_test.values,
                'predicted': predictions
            }).dropna()
            
            # Create scatter plot
            scatter_fig = px.scatter(
                results_df,
                x='actual',
                y='predicted',
                title=f'Actual vs Predicted {display_name}',
                labels={
                    'actual': f'Actual {display_name} ($)',
                    'predicted': f'Predicted {display_name} ($)'
                },
                trendline='lowess'
            )
            
            scatter_fig.update_layout(
                plot_bgcolor='white',
                title={'x': 0.5},
                xaxis=dict(
                    showline=True, 
                    linecolor='black',
                    mirror=True,
                    showgrid=True,
                    gridcolor='lightgray'
                ),
                yaxis=dict(
                    showline=True, 
                    linecolor='black',
                    mirror=True,
                    showgrid=True,
                    gridcolor='lightgray'
                ),
                margin=dict(l=50, r=50, t=60, b=50)
            )
            
            # Create time series plot
            time_fig = go.Figure()
            time_fig.add_trace(go.Scatter(
                x=results_df['date'],
                y=results_df['actual'],
                mode='markers',
                name='Actual',
                marker=dict(color='blue', size=8)
            ))
            time_fig.add_trace(go.Scatter(
                x=results_df['date'],
                y=results_df['predicted'],
                mode='markers',
                name='Predicted',
                marker=dict(color='red', size=8, symbol='x')
            ))
            
            time_fig.update_layout(
                title={'text': f'{display_name} Over Time: Actual vs Predicted', 'x': 0.5},
                xaxis_title='Date',
                yaxis_title=f'{display_name} ($)',
                plot_bgcolor='white',
                hovermode='x unified',
                xaxis=dict(
                    showline=True, 
                    linecolor='black',
                    mirror=True,
                    showgrid=True,
                    gridcolor='lightgray'
                ),
                yaxis=dict(
                    showline=True, 
                    linecolor='black',
                    mirror=True,
                    showgrid=True,
                    gridcolor='lightgray'
                ),
                margin=dict(l=50, r=50, t=60, b=50)
            )

            
            metrics_display = html.Div([
                html.H3("Model Performance Metrics", style={'color': '#2c3e50'}),
                html.P(f"R² Score: {r2:.3f}", style={'fontSize': '16px'}),
                html.P(f"Mean Absolute Error: ${mae:,.2f}", style={'fontSize': '16px'})
            ])
            
            return scatter_fig, time_fig, metrics_display
    
    def run(self):
        """Start the dashboard server"""
        self.app.run(mode='inline', height=1000)

In [21]:
# Initialize and run the dashboard
if __name__ == '__main__':
    dashboard = RetailDashboard(df)
    dashboard.run()